# 회귀 분석

In [1]:
# data 전처리

import pandas as pd
import numpy as np

data=pd.read_csv('Data_Science_Fields_Salary_Categorization.csv', index_col=0)
group_mapping = {
    "Data Engineer": [
        "Data Engineer", "Big Data Engineer", "Cloud Data Engineer", "Lead Data Engineer", 
        "Principal Data Engineer", "Data Engineering Manager", "ETL Developer", "Data Architect", 
        "Big Data Architect"
    ],
    "Data Scientist": [
        "Data Scientist", "Applied Data Scientist", "Principal Data Scientist", "Staff Data Scientist", 
        "Research Scientist", "AI Scientist", "Director of Data Science", "Head of Data Science", 
        "Data Science Manager", "Data Science Consultant", "Data Science Engineer"
    ],
    "Machine Learning Engineer": [
        "Machine Learning Engineer", "ML Engineer", "Machine Learning Developer", 
        "Machine Learning Scientist", "Applied Machine Learning Scientist", "Lead Machine Learning Engineer", 
        "Machine Learning Infrastructure Engineer", "Head of Machine Learning", "Machine Learning Manager", 
        "NLP Engineer"
    ],
    "Data Analyst": [
        "Data Analyst", "Business Data Analyst", "BI Data Analyst", "Financial Data Analyst", 
        "Marketing Data Analyst", "Product Data Analyst", "Lead Data Analyst", 
        "Principal Data Analyst", "Data Specialist", "Data Analytics Manager", 
        "Data Analytics Engineer", "Data Analytics Lead", "Analytics Engineer"
    ],
    "Computer Vision Engineer": [
        "Computer Vision Engineer", "3D Computer Vision Researcher", "Computer Vision Software Engineer"
    ]
}

def map_designation(designation):
    for group, titles in group_mapping.items():
        if designation in titles:
            return group
    return 'Other'

data['Group'] = data['Designation'].apply(map_designation)
data=data[data['Group'] != 'Other']
data_copy=data.copy()
data_copy['Salary_In_Rupees']=data['Salary_In_Rupees'].replace(',', '', regex=True).astype(float)
data_copy['Salary_Million_Won']=(data_copy['Salary_In_Rupees']*16.5/1000000).astype(int)
data=data_copy[['Working_Year','Experience','Employment_Status',
           'Employee_Location','Company_Location','Company_Size',
           'Remote_Working_Ratio','Group','Salary_Million_Won']]

# e-03 보다 크다면 과학적 표기법으로 쓰는 것이 아닌 일반 숫자 표시로 나타내도록
def conditional_format(x):
    if abs(x) < 1e-3:
        return f"{x:.3e}"  # 과학적 표기법
    else:
        return f"{x:.6f}"  # 일반 숫자 표시

# Pandas의 표시 형식에 조건부 포매팅 함수 적용
pd.options.display.float_format = conditional_format

In [29]:
import statsmodels.api as sm

# Categorical 변수를 수치형으로 변경(더미 변수화)
X = pd.get_dummies(data[['Group', 'Experience', 'Company_Size', 
                         'Remote_Working_Ratio', 'Employment_Status', 
                         'Employee_Location', 'Company_Location']], drop_first=True)
X_const = sm.add_constant(X)
y = data['Salary_Million_Won']

# 회귀 분석
lm = sm.OLS(y, X_const).fit()

# 결과 요약
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:     Salary_Million_Won   R-squared:                       0.563
Model:                            OLS   Adj. R-squared:                  0.484
Method:                 Least Squares   F-statistic:                     7.124
Date:                Sun, 08 Dec 2024   Prob (F-statistic):           5.10e-49
Time:                        19:32:21   Log-Likelihood:                -3300.2
No. Observations:                 596   AIC:                             6784.
Df Residuals:                     504   BIC:                             7188.
Df Model:                          91                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

## 회귀 분석 결과
- 경력(Experience)가 연봉에 가장 큰 영향을 미치는 변수로 나타난다.
- 대기업이 중소기업보다 높은 연봉을 제공한다.
- 미국(US) 직원의 연봉이 유의미하게 높다.

R-Squared는 0.563이지만 다중회귀분석이므로 변수의 수가 증가해 평가에 좋은 measure가 아니다.
변수의 수를 고려한 Adjusted R-Squared는 0.484로, 따라서 연봉(Salary)의 48.4%의 variance가 독립 변수들로 설명이 가능하다.



Experience_EX (Expert): 계수 158.35, p-value < 0.001
Experience_SE (Senior Level): 계수 61.23, p-value < 0.001
Experience_MI (Mid Level): 계수 32.63, p-value = 0.001
Remote_Working_Ratio(재택근무 비율): 계수 0.078, p-value > 0.05로 유의하지 않습니다.
Employment_Status(고용 상태):
고용 상태는 급여에 중요한 영향을 미치며, 특히 비정규직과 관련된 상태(FL, PT 등)는 급여를 낮추는 경향이 있습니다.
Employment_Status_PT(파트타임): -102.63 (p-value < 0.05)
Employment_Status_FL(계약직): -102.44 (p-value > 0.05)